# TITLE I

## Lib's

In [1]:
import pandas as pd
import re
import os

from dash import Dash
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash import dcc
from dash import html
import base64


## Dataset

In [2]:
#reading data
path = 'data'
lista = []
df = pd.DataFrame()

for f in os.listdir(path + "/"):
    frames = pd.read_csv(path + "/" + f, encoding = 'latin-1', sep = ';', low_memory = False)
    file = re.search(r'\d+', f).group(0)
    
    frames['file'] = file
    df = df.append(frames, ignore_index=True)

In [3]:
df.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,ilesos,ignorados,feridos,veiculos,file,latitude,longitude,regional,delegacia,uop
0,1012870.0,25/11/2008,Terça,16:00:00,DF,70,5,BRASILIA,Outras,Colisão traseira,...,2,0,1,2,2008,NaN,NaN,NaN,NaN,NaN
1,1023692.0,09/11/2008,Domingo,10:10:00,MA,316,284,BELA VISTA DO MARANHAO,Outras,Tombamento,...,1,0,9,1,2008,NaN,NaN,NaN,NaN,NaN
2,1027252.0,27/11/2008,Quinta,09:20:00,PE,101,47.6,IGARASSU,Falta de atenção,Colisão traseira,...,2,0,2,3,2008,NaN,NaN,NaN,NaN,NaN
3,1029670.0,11/01/2008,Sexta,22:00:00,MA,316,432.1,PERITORO,Ultrapassagem indevida,Colisão frontal,...,0,0,2,2,2008,NaN,NaN,NaN,NaN,NaN
4,1038085.0,07/05/2008,Quarta,07:05:00,RO,364,719.6,PORTO VELHO,Falta de atenção,Colisão Transversal,...,1,0,2,2,2008,NaN,NaN,NaN,NaN,NaN


In [4]:
# data type
df['data_inversa'] = df['data_inversa'].astype('datetime64')

print(df['data_inversa'].min())
print(df['data_inversa'].max())

2007-01-01 00:00:00
2021-06-30 00:00:00


In [5]:
# add new columns
df['month'] = df['data_inversa'].dt.month
df['year'] = df['data_inversa'].dt.year

df['time'] = df['horario'].str.slice(0, 2)

print(df['horario'].min(), df['horario'].max())

00:00:00 23:59:00


In [6]:
df.head(3).T

,0,1,2
id,1012870.0,1023692.0,1027252.0
data_inversa,2008-11-25 00:00:00,2008-09-11 00:00:00,2008-11-27 00:00:00
dia_semana,Terça,Domingo,Quinta
horario,16:00:00,10:10:00,09:20:00
uf,DF,MA,PE
br,70,316,101
km,5,284,47.6
municipio,BRASILIA,BELA VISTA DO MARANHAO,IGARASSU
causa_acidente,Outras,Outras,Falta de atenção
tipo_acidente,Colisão traseira,Tombamento,Colisão traseira


In [7]:
# getting only the target columns
df1 = df[['dia_semana', 'uf', 'br', 'km',
       'municipio', 'causa_acidente', 'tipo_acidente',
        'fase_dia', 
       'condicao_metereologica', 'tipo_pista', 
       'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos',
       'ignorados', 'feridos', 'file', 'month', 'year', 'time']]

In [8]:
#replacing some values
df1 = df1.copy(deep = True)

df1['dia_semana'] = [x.lower() for x in df1['dia_semana']]
df1['dia_semana'] = [x.replace('-feira', '') for x in df1['dia_semana']]

df1['causa_acidente'] = df1['causa_acidente'].str.lower()

dict_replace = {'ingestão de álcool e/ou substâncias psicoativas pelo pedestre' : 'ingestão de álcool',
               'ingestão de álcool ou de substâncias psicoativas pelo pedestre' : 'ingestão de álcool',
               'ingestão de álcool pelo condutor' : 'ingestão de álcool',
               'redutor de velocidade em desacordo' : 'velocidade incompatível',
               'dormindo' : 'condutor dormindo',
               'falta de atenção à condução' : 'falta de atenção',
               'defeito mecânico em veículo' : 'defeito mecânico no veículo',
               'mal súbito do condutor' : 'mal súbito'}

df1 = df1.replace({'causa_acidente' : dict_replace})


In [9]:
df1['dia_semana'].unique()

array(['terça', 'domingo', 'quinta', 'sexta', 'quarta', 'sábado',
       'segunda'], dtype=object)

# PLOTLY DASHBOARD

In [10]:
df1.columns

Index(['dia_semana', 'uf', 'br', 'km', 'municipio', 'causa_acidente',
       'tipo_acidente', 'fase_dia', 'condicao_metereologica', 'tipo_pista',
       'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos',
       'ignorados', 'feridos', 'file', 'month', 'year', 'time'],
      dtype='object')

In [11]:
df1['tipo_acidente'].unique()

array(['Colisão traseira', 'Tombamento', 'Colisão frontal',
       'Colisão Transversal', 'Capotamento', 'Colisão com objeto fixo',
       'Atropelamento de animal', 'Saída de Pista',
       'Queda de motocicleta / bicicleta / veículo',
       'Atropelamento de pessoa', 'Colisão lateral',
       'Colisão com bicicleta', 'Derramamento de Carga',
       'Colisão com objeto móvel', 'Danos Eventuais', 'Incêndio',
       'Atropelamento de Animal', 'Atropelamento de Pedestre',
       'Saída de leito carroçável', 'Colisão transversal',
       'Colisão com objeto estático', 'Queda de ocupante de veículo',
       'Engavetamento', 'Colisão com objeto em movimento',
       'Danos eventuais', 'Derramamento de carga', 'Colisão com objeto',
       'Colisão lateral mesmo sentido', 'Colisão lateral sentido oposto',
       'Eventos atípicos'], dtype=object)

In [12]:
# app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY],
#                 meta_tags = [{'name' : 'viewport',
#                               'content' : 'width=dice_width, initial-scale=1.0'}])

# app.layout = dbc.Container([
#     html.P(' '),
#     html.Div(id = '1st-line-separator', style = {'border' : '4px LightCoral solid'}),   
#         dbc.Card([
#             dbc.CardBody([
#                 html.H1('ACIDENTES RODOVIÁRIOS', style = {'color' : '#F8F9F9'},
#                    className = 'text-center'),
#                 html.H5('2008 à 2021', style = {'color' : '#F8F9F9'},
#                    className = 'text-center')
#                     ],  style={'border' : 'none'})
#                 ]),
        
#     html.P(' '),
#     html.Div([
#         html.Span('Dashboard referente a acidentes rodoviários registrados no períiodo de 2008 à 2021\ne disponibilizados pela Polícia Federal do Brasil'),
#         ], style = {'color' : '#A6ACAF', 'fontSize': 14, 'font-weight': 'light'},
#     className = 'text-center'),
#     html.P(' '),    
#     html.Div(id = '2st-line-separator', style = {'border' : '1px Lightpink solid'}),
#     html.P(' '),
#     dbc.Card([
#     html.P(' '),  
#     dbc.Row([
#         dbc.Col([
#             html.H5("Anos", style = {'color' : 'DimGrey', 'font-size' : 15, 'font-weight':'normal' })
#         ], width = {'size' : 1,'offset' : 0.5}),
#         dbc.Col([
#             dcc.RangeSlider(id='year-slider',
#                             marks = {str(i) : i for i in df['year'].unique()},
#                             count = 1,
#                             min = df1['year'].min(),
#                             max = df1['year'].max(),
#                             step = 1,
#                             value = [2015, 2021],
#                             className="margin9")
#         ],  width = {'size' : 10,'offset' : 0}),
#         ]),
#     html.P(' '),
#     dbc.Row([
#         dbc.Col([     
#             html.H5('Meses: ', style = {'color' : 'DimGrey', 'font-size' : 15, 'font-weight':'normal' })
#         ], width = {'size' : 1,'offset' : 0.5}),
        
#     dbc.Col([
#         dcc.Dropdown(id='month-picker',
#                             multi = True, value = [1, 2],
#                             options = [{'label' : x, 'value' : x} for x in sorted(df1['month'].unique()) if x != '(null)'])
#     ], width = {'size' : 4,'offset' : 0}),
#     dbc.Col([
#             html.H5('Dia da semana: ', style = {'color' : 'DimGrey', 'font-size' : 15, 'font-weight':'normal' })
#     ], width = {'size' : 1, 'offset' : 0}),
#     dbc.Col([dcc.Dropdown(id = 'week-day-picker',
#                             multi = True, value = 'domingo',
#                             options = [{'label' : x, 'value' : x} for x in sorted(df1['dia_semana'].unique()) if x != '(null)'])
#     ], width = {'size' : 4, 'offset' : 0})
#         ]),
#     html.Div(id = '20st-line-separator', style = {'border' : '0px Gainsboro solid'}),
#     ]),
#     dbc.Card([
#     html.P(' '),
#     dbc.Row([
#         dbc.Col([
#         html.H5('Estados: ', style = {'color' : 'DimGrey', 'font-size' : 15, 'font-weight':'normal' })
#     ], width = {'size' : 1, 'offset' : 0}),    
#     dbc.Col([
#         dcc.Dropdown(id='state-picker',
#                             multi = True, value = [1, 2],
#                             options = [{'label' : x, 'value' : x} for x in sorted(df1['uf'].unique()) if x != '(null)'])
#     ], width = {'size' : 9,'offset' : 0}),
#         html.P(' ')
#     ])
#         ]),
#     dbc.Col([
#             dcc.Graph(id='line-plot', figure = {},
#             style = {'border' : '1px Azure Solid'})
#         ], xs = 12, sm = 12, md = 12, lg = 9, xl = 9)

#     ])


# @app.callback(
#     Output('line-plot', 'figure'),
#     Input('year-slider', 'value')
# )
# def update_first_plot(year_selected):
#     df01 = df1[['year', 'mortos']].groupby(['year'], as_index = False).sum()
#     df01 = df1[df['year'].isin(year_selected)]
#     fig1 = px.line(df01, x ='Date', y = 'mortos', color = 'Symbols')
             
    

# if __name__ == '__main__':
#     app.run_server()

In [13]:
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY],
                meta_tags = [{'name' : 'viewport',
                              'content' : 'width=dice_width, initial-scale=1.0'}])

app.layout = dbc.Container([
    html.P(' '),
    html.Div(id = '1st-line-separator', style = {'border' : '4px LightCoral solid'}),   
        dbc.Card([
            dbc.CardBody([
                html.H1('ACIDENTES RODOVIÁRIOS', style = {'color' : '#F8F9F9'},
                   className = 'text-center'),
                html.H5('2008 à 2021', style = {'color' : '#F8F9F9'},
                   className = 'text-center')
                    ],  style={'border' : 'none'})
                ]),     
    html.P(' '),
    html.Div([
        html.Span('Dashboard referente a acidentes rodoviários registrados no períiodo de 2008 à 2021\ne disponibilizados pela Polícia Federal do Brasil'),
        ], style = {'color' : '#A6ACAF', 'fontSize': 14, 'font-weight': 'light'},
    className = 'text-center'),
    html.P(' '),    
    html.Div(id = '2st-line-separator', style = {'border' : '1px Lightpink solid'}),
    html.P(' '),
    
    dbc.Row([
        dbc.Col([
            html.H1('Ano da ocorrência', style = {'color' : '#A6ACAF', 'fontSize' : 10,
                                                    'font-weight' : 'light',
                                                    'backgroundColor' : 'orange'})
        ], width = {'size' : 2, 'offset' : 0}),
        dbc.Col([
            
            html.H1('Ano da ocorrência', style = {'color' : '#A6ACAF', 'fontSize' : 10, 'font-weight' : 'light',
                                             'backgroundColor' : 'red'})
            ], width = {'size' : 2,'offset' : 3}),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Slider(id = 'year-slider', min = 2007, max = 2021, value = 2007, step = 1,
                      marks = {i: i for i in range(2007, 2021)})
        ])
    ])
])

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


In [137]:
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY],
                meta_tags = [{'name' : 'viewport',
                              'content' : 'width=dice_width, initial-scale=1.0'}])


app.layout = dbc.Container([
    html.P(' '),
    html.Div(id = '1st-line-separator', style = {'border' : '4px LightCoral solid'}),   
        dbc.Card([
            dbc.CardBody([
                html.H2('ACIDENTES RODOVIÁRIOS', style = {'color' : '#F8F9F9'},
                   className = 'text-center'),
                html.H5('2008 à 2021', style = {'color' : '#F8F9F9'},
                   className = 'text-center')
                    ],  style={'border' : 'none'})
                ]),     
    html.P(' '),
    html.Div([
        html.Span('Dashboard referente a acidentes rodoviários registrados no período de 2008 à 2021\ne disponibilizados pela Polícia Federal do Brasil'),
        ], style = {'color' : '#A6ACAF', 'fontSize': 10, 'font-weight': 'light'},
    className = 'text-center'),
    html.P(' '),    
    html.Div(id = '2st-line-separator', style = {'border' : '2px #343a40 solid'}),
    html.P(' '),
    
    html.Div([
    dbc.Row([
        dbc.Col([
            html.H1('Ano da ocorrência', style = {'color' : 'white', 'fontSize' : 9,
                                                    'font-weight' : 'light',
                                                  'font-weight' : 'light'
                                                    }),
            ],width = {'size' :6}, className = 'col-ano-ocorrencia'),
        dbc.Col([
            html.H1('UF da ocorrência', style = {'color' : 'white', 'fontSize' : 9,
                                            'font-weight' : 'light',
                                            'font-weight' : 'light'}),
        ], width = {'size' : 5}, className = 'col-uf-ocorrencia')
    ], style = {'width' : '100%'}),
        
    dbc.Row([
        dbc.Col([
            dcc.Slider(min = 2007, max = 2021, value = 2007, step = 1,
                       className = 'year-slider',
                      marks = {str(i) : i for i in df['year'].unique()})
        ], width = {'size' : 6}),
        
        dbc.Col([
            dcc.Dropdown(options = ['SP', 'RJ', 'SE'],
                         style={'height': '20px', 
                                'width' : '95%',
                                'display': 'inline-block'},
                        className = 'dropdown-states'),
        ], width = {'size' : 5}),
         

        ], className = 'container-year-slider',
            style = {'width' : '100%'})
    ])
        
    
])
        

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/May/2022 10:11:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/May/2022 10:11:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 10:11:37] "GET /assets/01-style.css?m=1651756293.483545 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/May/2022 10:11:37] "GET /assets/01-style.css?m=1651756293.483545 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 10:11:37] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/May/2022 10:11:37] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 10:11:37] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/May/2022 10:11:37] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 10:11:37] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/May/2022 10:11:37] "GET /_dash-component-suites/dash/

In [15]:
df1.columns

Index(['dia_semana', 'uf', 'br', 'km', 'municipio', 'causa_acidente',
       'tipo_acidente', 'fase_dia', 'condicao_metereologica', 'tipo_pista',
       'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos',
       'ignorados', 'feridos', 'file', 'month', 'year', 'time'],
      dtype='object')

In [16]:
df.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,veiculos,file,latitude,longitude,regional,delegacia,uop,month,year,time
0,1012870.0,2008-11-25,Terça,16:00:00,DF,70,5,BRASILIA,Outras,Colisão traseira,...,2,2008,NaN,NaN,NaN,NaN,NaN,11,2008,16
1,1023692.0,2008-09-11,Domingo,10:10:00,MA,316,284,BELA VISTA DO MARANHAO,Outras,Tombamento,...,1,2008,NaN,NaN,NaN,NaN,NaN,9,2008,10
2,1027252.0,2008-11-27,Quinta,09:20:00,PE,101,47.6,IGARASSU,Falta de atenção,Colisão traseira,...,3,2008,NaN,NaN,NaN,NaN,NaN,11,2008,09
3,1029670.0,2008-11-01,Sexta,22:00:00,MA,316,432.1,PERITORO,Ultrapassagem indevida,Colisão frontal,...,2,2008,NaN,NaN,NaN,NaN,NaN,11,2008,22
4,1038085.0,2008-07-05,Quarta,07:05:00,RO,364,719.6,PORTO VELHO,Falta de atenção,Colisão Transversal,...,2,2008,NaN,NaN,NaN,NaN,NaN,7,2008,07
